In [101]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import importlib
import download_clean_data as dc
import ipca
import metrics
import kernel_regression as kr
importlib.reload(dc) 
importlib.reload(ipca)
importlib.reload(metrics)
importlib.reload(kr)

<module 'kernel_regression' from '/Users/matteoferrazzi/Documents/GitHub/ML-project-2/kernel_regression.py'>

In [20]:
folder_path = "/Users/matteoferrazzi/Documents/GitHub/ML-project-2/Data/monthly_data"

In [21]:
N = 100
data, ret = dc.download_clean_data(folder_path, 20150000, N)

In [22]:
x = []
for t in range(len(ret)-1):
    x.append(data[t].values.T@ret[t+1].values/len(ret[0]))

x_cov = np.sum([x[i].reshape((-1,1))@x[i].reshape((1,-1)) for i in range(len(x))], axis = 0)
eigval_x, eigvec_x = np.linalg.eig(x_cov)

idx = np.argsort(np.abs(eigval_x))[::-1]
sort_eigvec_x = eigvec_x[:,idx].real
k = 5
gamma_first = sort_eigvec_x[:,:k]

In [23]:
max_iter = 10
gamma, f_list = ipca.ipca(data, ret, gamma_first.copy(), max_iter)
W = np.eye(N)
W_list = [W]*(len(data)-1)
lambda1 = 0.1
lambda2 = 0.1
gamma_reg_w, f_list_reg_w = ipca.ipca_reg_w(data, ret, gamma_first.copy(), max_iter, lambda1, lambda2, W_list)

In [24]:
print(metrics.total_R_squared(ret, data, gamma, f_list),metrics.pred_R_squared(ret, data, gamma, f_list))

0.10452020486623592 0.000996034834962578


In [25]:
print(metrics.total_R_squared(ret, data, gamma_reg_w, f_list_reg_w),metrics.pred_R_squared(ret, data, gamma_reg_w, f_list_reg_w))

0.07980347398664922 0.0005313654697319814


In [92]:
Omega1=np.eye((len(data)-1)*N)
Omega2=np.eye(N)
tk = 1
l = 1
f_list_kr, v_kr = kr.kernel_regression(data, ret, f_list, lambda1, lambda2, Omega1, Omega2, max_iter, N, tk, l)

0


KeyboardInterrupt: 

In [93]:
data2 = data.copy()
data2 = np.array(np.array(data2).reshape(72*100,94)) #flatten data, build X
m_hat = 10
K = kr.K_LR(data2, tk, l)
L, B = kr.pivoted_chol(K, m_hat)

1
2
3
4
5
6
7
8
9


In [95]:
f_list_kr_LR, v_kr_LR = kr.kernel_regression_LR(data, data2, K, B, ret, f_list, lambda1, lambda2, Omega2, max_iter, m_hat, tk, l)

0


KeyboardInterrupt: 